In [1]:
from yolo import YOLO
from utils import get_annotations_images, LabelEncoder, read_labels
from config import Config
from networkfactory import NetworkFactory
from batchgenerators import batch_generator, batch_generator_inmemory
from filter_dataset import filter_datasaet

Using TensorFlow backend.


In [2]:
annotation_folder = r'.\annotations'
images_folder = r'.\images'

annotations, images = get_annotations_images(annotation_folder, images_folder)
print(len(annotations))

5011


In [3]:
images_dir = r'./images'
annotations_dir = r'./annotations'
image_sets_dir = r'./imagesets'

wanted_sets = ['bicycle', 'bus', 'car', 'motorbike', 'person']

annotations_sdc, images_sdc = filter_datasaet(images_dir, annotations_dir, image_sets_dir, wanted_sets)
print(len(annotations_sdc))

2724


In [14]:
labels_dir = "./labels.txt"
cfg_path = r"./cfg/mobilenetyolov2.cfg"
weights = r"./weights/mobilenetyolov2_take2"
weights = r"./weights/mobilenetyolov2-checkpoint-50"


cfg = Config(cfg_path)
encoder = LabelEncoder(read_labels(labels_dir)[0])
networkfactory = NetworkFactory()

#yolo = YOLO(cfg, encoder, networkfactory)
yolo = YOLO(cfg, encoder, networkfactory, weights)

C:\Users\Gencho\Anaconda3\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [ ]:
epochs = 250

from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping

def train(self, generator, annotations, images, epochs, checkpoint_period=None, early_stopping=False):
    # gen = tf.keras.preprocessing.image.ImageDataGenerator()

    annotations_train, annotations_val, images_train, images_val = train_test_split(annotations, images,
                                                                                    test_size=0.1)

    gen_train = generator(annotations_train, images_train, self.cfg,
                          self.networkfactory.get_normalizer(self.cfg),
                          self.encode_y_true_from_annotation)

    gen_val = generator(annotations_val, images_val, self.cfg,
                        self.networkfactory.get_normalizer(self.cfg),
                        self.encode_y_true_from_annotation)

    callbacks = []

    if checkpoint_period:
        modelname = self.cfg.get("net")
        filepath = "./weights/" + modelname + "yolov2-checkpoint-{epoch:02d}"

        checkpoint = ModelCheckpoint(filepath=filepath, period=checkpoint_period)
        callbacks.append(checkpoint)

    if early_stopping:
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        #callbacks.append(es)

    h = self.model.fit_generator(gen_train, steps_per_epoch=len(images_train) // self.cfg.get('batch_size'),
                                 ##validation_data=gen_val,
                                 ##validation_steps=len(images_val) // self.cfg.get('batch_size'),
                                 epochs=epochs,
                                 callbacks=callbacks)

    # TODO: Give a summary

train(yolo, batch_generator_inmemory, annotations[:300], images[:300] , epochs, 5, True)
#yolo.train(batch_generator, annotations, images, epochs, 5, True)

Epoch 1/250


In [5]:
yolo.save(weights)